In [1]:
import httplib2
import os

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

from pandas import DataFrame, concat, to_datetime, to_numeric
import pandas as pd

import json

# If modifying these scopes, delete your previously saved credentials
# at ~/.credentials/sheets.googleapis.com-python-quickstart.json
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
CLIENT_SECRET_FILE = 'gsheets_secret.json'
APPLICATION_NAME = 'solarpunktravel backend'

spreadsheetId = '1FHoBYKs-kVpmkmyUi1BA5_qmygJUo0UfiAayc6mqgcc'

In [2]:
def get_credentials():
    """Gets valid user credentials from storage.
    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.
    Returns:
        Credentials, the obtained credential.
    """
    credential_path = 'sheets.googleapis.com-solarpunktravel-backend.json'

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flags = tools.argparser.parse_args(args=[])
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        credentials = tools.run_flow(flow, store, flags)
        print('Storing credentials to ' + credential_path)
    return credentials

#https://stackoverflow.com/questions/38245714/get-list-of-sheets-and-latest-sheet-in-google-spreadsheet-api-v4-in-python
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                'version=v4')
service = discovery.build('sheets', 'v4', http=http,
                          discoveryServiceUrl=discoveryUrl)

In [3]:
sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheetId).execute()
sheets = sheet_metadata.get('sheets', '')
sheet_names = [s.get("properties", {}).get("title", "Sheet1") for s in sheets]


result = service.spreadsheets().values().get(
    spreadsheetId=spreadsheetId, range="A:J").execute()
raw_data = result.get('values', [])
download_data = DataFrame.from_records([r + [''] * (len(raw_data[0])-len(r)) for r in raw_data[1:]],columns=raw_data[0])

In [4]:
###
#  preliminary injection of all the batch downloaded data
###

df = download_data.copy()

with open(f"hub_umap_urls.json", "r") as infile:
    city_urls = json.load(infile)
    
city_list =list(city_urls.keys())
for c in city_list:
    try:
        camp = pd.read_csv(f'hub_gpx/{c}/camp_db.csv')
        camp_new = pd.DataFrame()
        camp_new[['name','lat','lon','journey']]=camp[['4','1','0','journey']]
        camp_new['hub']=c
        camp_new['type']='shelter'
        camp_new['subtype']='public_camp'

        food = pd.read_csv(f'hub_gpx/{c}/food_db.csv')
        food_new = pd.DataFrame()
        food_new[['name','lat','lon','url']]=food[['name','lat','lon','url']]
        food_new['hub']=c
        food_new['type']='resource'
        food_new['subtype']='food_coop'

        bike = pd.read_csv(f'hub_gpx/{c}/bike_db.csv')
        bike_new = pd.DataFrame()
        bike_new[['name','lat','lon','url']]=bike[['name','lat','lon','url']]
        bike_new['hub']=c
        bike_new['type']='resource'
        bike_new['subtype']='bike_coop'

        gear = pd.read_csv(f'hub_gpx/{c}/gear_db.csv')
        gear_new = pd.DataFrame()
        gear_new[['name','lat','lon','url']]=gear[['name','lat','lon','url']]
        gear_new['hub']=c
        gear_new['type']='resource'
        gear_new['subtype']='used_gear_store'

        rout = pd.read_csv(f'hub_gpx/{c}/rout_db.csv')
        rout_new = pd.DataFrame()
        rout_new[['name','description','image','url','journey','lat','lon']]=rout[['name','description','image','url','journey','lat','lon']]
        rout_new['hub']=c
        rout_new['type']='route'
        rout_new['subtype']='ridewithgps'

        to_post = pd.concat([camp_new,food_new,bike_new,gear_new,rout_new]).fillna('')[['hub','type','subtype','name','url','description','lat','lon','image','journey']]
        df = pd.concat([df,to_post])
    except:
        print(f"{c} failed")

df = df.drop_duplicates()
body = {
    "majorDimension": "ROWS",
    "values": [
        df.columns.tolist(),
        ] + df.values.tolist(),
}
service.spreadsheets().values().update(
    spreadsheetId=spreadsheetId,
    range='A:J',
    body=body,
    valueInputOption='USER_ENTERED'
).execute()

detroit failed


{'spreadsheetId': '1FHoBYKs-kVpmkmyUi1BA5_qmygJUo0UfiAayc6mqgcc',
 'updatedRange': 'resources!A1:J3170',
 'updatedRows': 3170,
 'updatedColumns': 10,
 'updatedCells': 31700}